In [4]:
import pandas as pd

def get_data_by_keys(raw_data, fiscal_year, has_withdrawn = False):
    standard_keys = ['CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'DECISION_DATE', 'VISA_CLASS','EMPLOYMENT_START_DATE','EMPLOYMENT_END_DATE',
        'EMPLOYER_NAME', 'EMPLOYER_CITY', 'EMPLOYER_STATE', 'EMPLOYER_POSTAL_CODE', 'JOB_TITLE', 'SOC_CODE', 'TOTAL_WORKERS']
    if has_withdrawn:
        standard_keys.append("WITHDRAWN")
    data = raw_data[standard_keys]
    return data
        
    
def process_LCA_data(file_name, fiscal_year, has_withdrawn = False):
    
    # step 1: read LCA data
    # LCA_data = pd.read_excel(f'../LCA_data/data/H-1B_Case_Data_FY{fiscal_year}.xlsx', index_col= None)
    LCA_data = pd.read_excel(file_name, index_col= None)
    # LCA_data = LCA_data[['CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'DECISION_DATE', 'VISA_CLASS','EMPLOYMENT_START_DATE','EMPLOYMENT_END_DATE',
    #     'EMPLOYER_NAME', 'EMPLOYER_CITY', 'EMPLOYER_STATE', 'EMPLOYER_POSTAL_CODE', 'JOB_TITLE', 'SOC_CODE', 'TOTAL_WORKERS']]
    LCA_data = get_data_by_keys(LCA_data, fiscal_year, has_withdrawn=has_withdrawn)
    # step 2: filter out denied LCA
    LCA_data = LCA_data[LCA_data['CASE_STATUS'] == 'CERTIFIED']
    if has_withdrawn:
        LCA_data = LCA_data[LCA_data['WITHDRAWN'] == 'N']
    
    # step 3: LCA certified date should be between Jan and Apr
    LCA_certified_begin_date = pd.to_datetime(f'{fiscal_year}0101', format = '%Y-%m-%d')
    LCA_certified_end_date = pd.to_datetime(f'{fiscal_year}0331', format = '%Y-%m-%d')
    LCA_data = LCA_data[(LCA_data['DECISION_DATE'] >= LCA_certified_begin_date) & (LCA_data['DECISION_DATE'] <= LCA_certified_end_date)]
    # step 4: LCA employment date should be between 5 and 6 months after being certified
    LCA_data.loc[:,'LCA_employment_begin_date'] = LCA_data['DECISION_DATE'] + pd.DateOffset(months=5)
    LCA_data.loc[:,'LCA_employment_end_date'] = LCA_data['DECISION_DATE'] + pd.DateOffset(months=6)
    LCA_data = LCA_data[(LCA_data['EMPLOYMENT_START_DATE'] >= LCA_data['LCA_employment_begin_date']) & (LCA_data['EMPLOYMENT_START_DATE'] <= LCA_data['LCA_employment_end_date'])]
    
    return LCA_data



In [5]:
def merge_2009_data(case_data, Icert_data):
    case_data = process_LCA_data(case_data, 2009, has_withdrawn = True)
    Icert_data = process_LCA_data(Icert_data, 2009, has_withdrawn = False)
    # merge two dataframes
    merged_data = pd.concat([case_data, Icert_data], axis = 0)
    print(f'2009 merged data shape: {merged_data.shape}')
    print(f'2009 case data shape: {case_data.shape}')
    print(f'2009 Icert data shape: {Icert_data.shape}')
    return merged_data

In [10]:
for years in [2008, 2009, 2014, 2015, 2016, 2017]:
    print(f'Processing data for fiscal year {years}')
    if years in [2008, 2009]:
        has_withdrawn = True
    else:
        has_withdrawn = False
    case_file = f'./data/H-1B_Case_Data_FY{years}.xlsx'
    processed_file = f'./processed_data/H-1B_Case_Data_FY{years}_processed.xlsx'
    processed_data = process_LCA_data(case_file, years, has_withdrawn = has_withdrawn)
    # save processed data
    processed_data.to_excel(processed_file, index = True)

Processing data for fiscal year 2008
Processing data for fiscal year 2009
Processing data for fiscal year 2014
Processing data for fiscal year 2015


/Users/wangyanze/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:339: UserWarning: Parsing '15/04/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return cls._from_sequence_not_strict(scalars, dtype=dtype, copy=copy)
/Users/wangyanze/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:339: UserWarning: Parsing '15/06/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return cls._from_sequence_not_strict(scalars, dtype=dtype, copy=copy)
/Users/wangyanze/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:339: UserWarning: Parsing '19/03/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return cls._from_sequence_not_strict(scalars, dtype=dtype, copy=copy)
/Users/wangyanze/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:339: U

Processing data for fiscal year 2016
Processing data for fiscal year 2017


In [2]:
# # group by employer name
# endata = LCA_data.groupby('EMPLOYER_NAME').sum().sort_values(by = 'EMPLOYER_NAME', ascending = False)
# # save to excel
# endata.to_excel(f'test.xlsx', index = True)

In [1]:
# from fuzzywuzzy import process
# import tqdm
# # Extract unique company names
# company_names = LCA_data['EMPLOYER_NAME'].unique()

# # Create a mapping for similar names
# def get_best_match(name, name_list, threshold=90):
#     match, score = process.extractOne(name, name_list)
#     return match if score >= threshold else name

# print(company_names.shape)
# # Apply fuzzy matching
# unique_names = []
# name_mapping = {}
# for name in tqdm.tqdm(company_names):
#     if len(unique_names) == 0:
#         unique_names.append(name)
#         name_mapping[name] = name
#         continue
    
#     best_match = get_best_match(name, unique_names)
#     if best_match not in unique_names:
#         unique_names.append(best_match)
#     name_mapping[name] = best_match
    
#     # print(f'{name} -> {best_match}')
# #     if best_match not in unique_names:
# #         unique_names.append(best_match)
# #     name_mapping[name] = best_match

# # # Replace original names with the best matches
# # LCA_data['company_name'] = LCA_data['EMPLOYER_NAME'].map(name_mapping)

# # # Now you can group by the standardized company names
# # fuzzy_LCA_data = LCA_data.groupby('company_name').sum()